In [109]:
# -*- coding: utf-8 -*-
import os, sys
from os.path import dirname, join
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [110]:
df = pd.read_csv("../databases/risco_credito.csv")
previsores = df.iloc[:, 0:4].values
classe = df.iloc[:, 4].values

In [111]:
# ATTRIBUTES
# ----------

for c in df.columns:
    print(
        "{0}\nTotal: {1}\n".format(df[c].value_counts(), 
        df['garantias'].value_counts().values.sum()))

boa             5
desconhecida    5
ruim            4
Name: historia, dtype: int64
Total: 14

alta        8
moderado    3
baixa       3
Name: divida, dtype: int64
Total: 14

nenhuma     11
adequada     3
Name: garantias, dtype: int64
Total: 14

acima_35    7
15_35       4
0_15        3
Name: renda, dtype: int64
Total: 14

alto        6
baixo       5
moderado    3
Name: c#risco, dtype: int64
Total: 14



In [112]:
# REGRA HISTORIA DE CRÉDITO - ATTRIBUTE BOA

attr_boa = df.iloc[:, 0:2].loc[df['historia'] == 'boa']
total_attr_divida_boa = pd.DataFrame(attr_boa)['divida'].value_counts().values.sum()

max_value_divida_boa = total_attr_divida_boa - pd.DataFrame(attr_boa)['divida'].value_counts().values.max()

max_value_historia_boa = pd.DataFrame(attr_boa)['historia'].value_counts().values.max()
error_boa = max_value_divida_boa / max_value_historia_boa

for c in pd.DataFrame(attr_boa).columns:
    print('{0}'.format(pd.DataFrame(attr_boa)[c].value_counts()))

print("boa: {0} divida: {1} errors: {2}"
    .format(max_value_historia_boa, max_value_divida_boa, error_boa))

boa    5
Name: historia, dtype: int64
alta        3
moderado    1
baixa       1
Name: divida, dtype: int64
boa: 5 divida: 2 errors: 0.4


In [113]:
# REGRA HISTORIA DE CRÉDITO - ATTRIBUTE DESCONHECIA

attr_desc = df.iloc[:, 0:2].loc[df.historia == 'desconhecida']
total_attr_divida_desc = pd.DataFrame(attr_desc)['divida'].value_counts().values.sum()

max_value_divida_desc = total_attr_divida_desc - pd.DataFrame(attr_desc)['divida'].value_counts().values.max()

max_value_desc = pd.DataFrame(attr_desc)['historia'].value_counts().values.max()
error_desc = max_value_divida_desc / max_value_desc

for c in pd.DataFrame(attr_desc).columns:
    print('{0}'.format(pd.DataFrame(attr_desc)[c].value_counts()))

print("desconhecida: {0} divida: {1} errors: {2}"
    .format(max_value_desc, max_value_divida_desc, error_desc))

desconhecida    5
Name: historia, dtype: int64
baixa       2
alta        2
moderado    1
Name: divida, dtype: int64
desconhecida: 5 divida: 3 errors: 0.6


In [114]:
# REGRA HISTORIA DE CRÉDITO - ATTRIBUTE RUIM
attr_ruim = df.iloc[:, 0:2].loc[df.historia == 'ruim']
total_attr_divida_ruim = pd.DataFrame(attr_ruim)['divida'].value_counts().values.sum()

max_value_divida_ruim = total_attr_divida_ruim - pd.DataFrame(attr_ruim)['divida'].value_counts().values.max()

max_value_ruim = pd.DataFrame(attr_ruim)['historia'].value_counts().values.max()
error_ruim = max_value_divida_ruim / max_value_ruim

for c in pd.DataFrame(attr_ruim).columns:
    print('{0}'.format(pd.DataFrame(attr_ruim)[c].value_counts()))

print("desconhecida: {0} divida: {1} errors: {2}"
    .format(max_value_ruim, max_value_divida_ruim, error_ruim))


ruim    4
Name: historia, dtype: int64
alta        3
moderado    1
Name: divida, dtype: int64
desconhecida: 4 divida: 1 errors: 0.25


In [115]:
# REGRA HISTORIA DE CRÉDITO - TOTAL DE ERROS
print((error_boa + error_desc + error_ruim) / df.historia.value_counts().values.sum())

14


In [116]:
# Árvores de decisão não podem ser categoricos
labelencoder = LabelEncoder()
previsores[:, 0] = labelencoder.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder.fit_transform(previsores[:, 1])
previsores[:, 2] = labelencoder.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder.fit_transform(previsores[:, 3])

In [117]:
decisionTree = DecisionTreeClassifier(criterion="entropy")
"""
  Attributes
  -----------
    feature_importances_ : classificar a importância de cada atributo
  
  Methods
  -------
    export_graphviz : dot_data : string
      representação gráfica da árvore de decisção
"""
decisionTree = decisionTree.fit(previsores, classe)

In [118]:
# export_graphviz(
#     decisionTree,
#     out_file=os.path.abspath(join(dirname(os.getcwd()), 'python/tree_credit_risk.dot')),
#     feature_names=["historia", "divida", "garantias", "renda"],
#     class_names=["alto", "moderado", "baixo"],
#     filled=True,
#     leaves_parallel=True,
# )

In [119]:
result = decisionTree.predict([[0, 0, 1, 2], [3, 0, 0, 0]])